In [46]:
!pip install transformers

In [47]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

In [48]:
!pip install gdown
import gdown

# Setting the URl path to the folder of JB_hackathon, already done for you
file_id = '1IEbKRSiRzt8mJcXnLYAJim7Vj4m4-_Z6'
url = f'https://drive.google.com/uc?id={file_id}'

# specifying the output directory name
output = 'audio_data.zip'

gdown.download(url, output, quiet=False)

! mkdir audio_data
! unzip audio_data.zip -d audio_data

Downloading...
From (original): https://drive.google.com/uc?id=1IEbKRSiRzt8mJcXnLYAJim7Vj4m4-_Z6
From (redirected): https://drive.google.com/uc?id=1IEbKRSiRzt8mJcXnLYAJim7Vj4m4-_Z6&confirm=t&uuid=95eb9ef1-4771-40a6-839f-312b21e55867
To: /Users/fabianjaeger/Developer/d-fakeit/audio_data.zip
100%|██████████| 753M/753M [00:06<00:00, 109MB/s]  


Archive:  audio_data.zip
   creating: audio_data/all/
  inflating: audio_data/__MACOSX/._all  
  inflating: audio_data/all/72K6TSQ829.wav  
  inflating: audio_data/__MACOSX/all/._72K6TSQ829.wav  
  inflating: audio_data/all/O7O51MZU4N.wav  
  inflating: audio_data/__MACOSX/all/._O7O51MZU4N.wav  
  inflating: audio_data/all/1PSFC9GSK4.wav  
  inflating: audio_data/__MACOSX/all/._1PSFC9GSK4.wav  
  inflating: audio_data/all/1JGXELMHOF.wav  
  inflating: audio_data/__MACOSX/all/._1JGXELMHOF.wav  
  inflating: audio_data/all/VXC2DB72R7.wav  
  inflating: audio_data/__MACOSX/all/._VXC2DB72R7.wav  
  inflating: audio_data/all/MMPJO0MZ0N.wav  
  inflating: audio_data/__MACOSX/all/._MMPJO0MZ0N.wav  
  inflating: audio_data/all/UMM0YIUKNH.wav  
  inflating: audio_data/__MACOSX/all/._UMM0YIUKNH.wav  
  inflating: audio_data/all/WML4D26C67.wav  
  inflating: audio_data/__MACOSX/all/._WML4D26C67.wav  
  inflating: audio_data/all/F3AJRF7TJG.wav  
  inflating: audio_data/__MACOSX/all/._F3AJRF7TJG.wa

In [49]:
# !pip install datasets
from datasets import load_dataset
import os 

# dataset = load_dataset("audiofolder", data_dir="./audio_data/all/")

def get_path_names(directory):
    extension = "wav"
    file_names = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                # file_names.append(file)
                file_names.append(os.path.join(root, file))
    return file_names

file_names = get_path_names("data/audio_data/all")
print(file_names)
#dataset = load_dataset("<username>/my_dataset")

# audio_sample = "./audio_data/all/0D2XANLNWB.wav"

['data/audio_data/all/72K6TSQ829.wav', 'data/audio_data/all/O7O51MZU4N.wav', 'data/audio_data/all/1PSFC9GSK4.wav', 'data/audio_data/all/1JGXELMHOF.wav', 'data/audio_data/all/VXC2DB72R7.wav', 'data/audio_data/all/MMPJO0MZ0N.wav', 'data/audio_data/all/UMM0YIUKNH.wav', 'data/audio_data/all/WML4D26C67.wav', 'data/audio_data/all/F3AJRF7TJG.wav', 'data/audio_data/all/1QJIZTDX38.wav', 'data/audio_data/all/G11VNYE1HH.wav', 'data/audio_data/all/0D9KDY33VL.wav', 'data/audio_data/all/VLGVUDJ90Q.wav', 'data/audio_data/all/Y6K2JU2H4B.wav', 'data/audio_data/all/ECGD9KF3VX.wav', 'data/audio_data/all/GT37129NRX.wav', 'data/audio_data/all/XXGLDEB0PE.wav', 'data/audio_data/all/PYVE80HN4Y.wav', 'data/audio_data/all/8R488ER0ZH.wav', 'data/audio_data/all/65YME8J8IE.wav', 'data/audio_data/all/NPVTQJD50W.wav', 'data/audio_data/all/RL9JXFY2JE.wav', 'data/audio_data/all/FSO4MWKKAR.wav', 'data/audio_data/all/HEHXM9668B.wav', 'data/audio_data/all/FPLK2Z9M1O.wav', 'data/audio_data/all/FJDGLG7POQ.wav', 'data/audio

In [50]:
# dataset = load_dataset("audio", data_files=file_names)

from datasets import Dataset, Audio
  
ds = Dataset.from_dict({"audio": file_names}).cast_column("audio", Audio())

In [51]:
!pip install librosa soundfile

from librosa import resample

In [52]:
print(ds.shape)
print(ds[0])
import tqdm
import torch


def resample_audio(audio, sr):
    audio_sample = ds[0]["audio"]
    waveform = audio_sample["array"]
    sampling_rate = audio_sample["sampling_rate"] 
    return resample(waveform, orig_sr=sampling_rate, target_sr=sr)

num_samples = ds.shape[0]

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

for i in range(ds.__len__()):
    audio_sample = ds[i]["audio"]
    path = os.path.splitext(os.path.basename(audio_sample["path"]))[0]
    # print(path)
    waveform = audio_sample["array"]
    sampling_rate = audio_sample["sampling_rate"]
    # print(waveform.shape, sampling_rate)
    resampled = resample(waveform, orig_sr=sampling_rate, target_sr=16000) 
    # input_features = processor(
    #     resampled, sampling_rate=16000, return_tensors="pt"
    # ).input_features
    feature_ext=processor.feature_extractor(resampled, sampling_rate=16000, return_tensors="pt")
    feature_ext = feature_ext["input_features"]
    print(feature_ext)
    
    save_path = "data/embeddings_tensors"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    torch.save(feature_ext, os.path.join(save_path, f"{path}.pt"))


(400, 1)
{'audio': {'path': 'data/audio_data/all/72K6TSQ829.wav', 'array': array([0.00000000e+00, 2.49951088e-10, 1.97288824e-10, ...,
       5.09913832e-07, 4.37271751e-07, 3.31357427e-07]), 'sampling_rate': 44100}}
tensor([[[-0.7407, -0.5774,  0.0641,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.4683,  0.1126,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.5157,  0.1149,  ..., -0.7407, -0.7407, -0.7407],
         ...,
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407]]])
tensor([[[-0.5593, -0.5593,  0.0803,  ..., -0.5593, -0.5593, -0.5593],
         [-0.5593, -0.5593,  0.1090,  ..., -0.5593, -0.5593, -0.5593],
         [-0.5593, -0.5593,  0.1194,  ..., -0.5593, -0.5593, -0.5593],
         ...,
         [-0.5593, -0.5593, -0.5593,  ..., -0.5593, -0.5593, -0.5593],
         [-0.5593, -0.5593, -0.5593,  ..., 

In [53]:
test_emb = torch.load("data/embeddings_tensors/0D2XANLNWB.pt")
print(test_emb)

tensor([[[-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347],
         [-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347],
         [-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347],
         ...,
         [-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347],
         [-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347],
         [-0.7347, -0.7347, -0.7347,  ..., -0.7347, -0.7347, -0.7347]]])


In [54]:
audio_sample = ds[0]["audio"]
waveform = audio_sample["array"]
sampling_rate = audio_sample["sampling_rate"]

# Resample waveform to 16000 Hz
waveform_resampled = resample(waveform, orig_sr=sampling_rate, target_sr=16000)

# Now use the resampled waveform with the processor
input_features = processor(
    waveform_resampled, sampling_rate=16000, return_tensors="pt"
).input_features

# Continue with the rest of your code...

In [55]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

In [56]:
# input_features = processor(
#     waveform, sampling_rate=sampling_rate, return_tensors="pt"
# ).input_features

feature_ext=processor.feature_extractor(waveform_resampled, sampling_rate=16000, return_tensors="pt")

print(feature_ext)
# Generate token ids
# predicted_ids = model.generate(input_features)

# Decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

# transcription[0]

{'input_features': tensor([[[-0.7407, -0.5774,  0.0641,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.4683,  0.1126,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.5157,  0.1149,  ..., -0.7407, -0.7407, -0.7407],
         ...,
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407],
         [-0.7407, -0.7407, -0.7407,  ..., -0.7407, -0.7407, -0.7407]]])}


In [57]:
feature_ext['input_features'].size()

torch.Size([1, 80, 3000])